In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# alle excel-tabellen in ein df einlesen
df = pd.DataFrame()
for i in range(6):
    df = pd.concat([df,pd.read_excel(f'../raw/Plakate_1945-90_ImportCBS_Teil{i+1}.xlsx', dtype={'ERJ':'str'})], ignore_index=True)

# alle zeilen löschen, in denen ERJ nicht besetzt ist
df.drop(df[pd.isna(df.ERJ)].index, inplace=True)

# die tabelle mit den ergänzten jahreswerten als Df 'jahre' einlesen und an das groß df anhängen
jahre = pd.read_excel('../raw/Plakate_1945-90_ERJ-fehlt.xlsx', dtype={'ERJ':'str'})
df = pd.concat([df,jahre], ignore_index=True)
df.info()

# bekannte kleinere datenfehler korrigieren
df.loc[df.Sprache == 'ger;hun', 'Sprache'] = 'ger/1hun'
df.loc[df.IMD == ' T', 'IMD'] = 'T'
df.loc[df.ERJ == '1950-60?', 'ERJ'] = '1950-1960?'
df.loc[df.Trägermaterial == 'TB-Gewebe', 'Trägermaterial'] = 'TB-gewebe'

df.Form = df.Form.str.strip()
df.Erscheinungsort = df.Erscheinungsort.str.strip()
df.ERJ = df.ERJ.str.strip()

df.loc[df.Form == "!41983807!", 'Form'] = "!041983807!"
df.loc[df.Form == "!40292908!", 'Form'] = "!040292908!"
df.loc[df.Form == "!40296709!", 'Form'] = "!040296709!"
df.loc[df.Form == "!41354672!", 'Form'] = "!041354672!"



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15662 entries, 0 to 15661
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   IMD              15662 non-null  object
 1   ERJ              15662 non-null  object
 2   Signatur         15662 non-null  object
 3   Format           15662 non-null  object
 4   Anzahl           15657 non-null  object
 5   Form             1706 non-null   object
 6   Anmerkung        5585 non-null   object
 7   Internvermerk    1998 non-null   object
 8   Trägermaterial   109 non-null    object
 9   Zustand          6039 non-null   object
 10  kons_bemerkung   7559 non-null   object
 11  Altsignatur      8917 non-null   object
 12  Ländercode       15662 non-null  object
 13  Sprache          15632 non-null  object
 14  Erscheinungsort  15584 non-null  object
dtypes: object(15)
memory usage: 1.8+ MB


In [3]:
# nicht vergebene signaturen ermitteln
# df[pd.isna(df['ERJ'])].to_excel('../raw/Plakate_1945-90_ERJ-fehlt.xlsx', index=False)
# df.Signatur.value_counts()
# df[df.Erscheinungsort == '[Ort nicht ermittelbar]']
# df[pd.isna(df.Erscheinungsort) | (df.Erscheinungsort == '[Ort nicht ermittelbar]') ]
# df[(df.Signatur == '1945-90/982') | (df.Signatur == '1945-90/13563') | (df.Signatur == '1945-90/914')].to_excel('../raw/Plakate_1945-90_Signatur-doppelt.xlsx', index=False)
signaturliste = [int(x.split('-')[0]) for x in df.Signatur.str[8:].to_list()]
signaturliste.sort()
for element in range(len(signaturliste)):
    if element > 0:
        if signaturliste[element-1] < signaturliste[element]-1:
            print(signaturliste[element-1],signaturliste[element])

    


3713 3718


In [18]:
df.Form.value_counts()

!041543718!    798
!040465810!    500
!041983807!    272
!944257690!     59
!041921550!     24
!040292908!     15
!04071280X!     13
!04324081X!     12
!040469026!      5
!040082407!      3
!942218981!      2
!040296709!      1
!041354672!      1
!941865975!      1
Name: Form, dtype: int64

In [3]:
def datensatz_schreiben(row):
    pica = "0500 Pa\n"

    if row.IMD == 'T':
        pica += "0501 Text$btxt\n"
    elif row.IMD == 'T/B':
        pica += "0501 Text$btxt\n0501 unbewegtes Bild$bsti\n"
    elif row.IMD == 'B':
        pica += "0501 unbewegtes Bild$bsti\n"
    
    pica += f"""0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy\n"""

    if len(str(row.ERJ)) == 4:
        # nur ein datum
        pica += f"1100 {row.ERJ}\n1110 *{row.ERJ}$4ezth\n"
    elif len(str(row.ERJ)) == 5 and str(row.ERJ.endswith('?')) == 'True':
        # ein jahr mit fragezeichen
        pica += f"1100 {row.ERJ[:4]}$n[{row.ERJ}]\n1110 {row.ERJ}*{row.ERJ[:4]}$4ezth\n"
    elif len(str(row.ERJ)) == 9:
        # zwei daten
        pica += f"1100 {row.ERJ[:4]}$n[zwischen {row.ERJ[:4]} und {row.ERJ[5:]}]\n1110 zwischen {row.ERJ[:4]} und {row.ERJ[5:]}*{row.ERJ}$a{row.ERJ[:4]}$b{row.ERJ[5:]}$4ezth\n"
    elif len(str(row.ERJ)) == 10:
        # zwei daten mit fragezeichen
        pica += f"1100 {row.ERJ[:4]}$n[zwischen {row.ERJ[:4]} und {row.ERJ[5:]}]\n1110 zwischen {row.ERJ[:4]} und {row.ERJ[5:]}*{row.ERJ[:9]}$a{row.ERJ[:4]}$b{row.ERJ[5:9]}$4ezth\n"
    else:
        pica += f"\n\n1100 FEHLER {row.Signatur}\n\n"
    
    if pd.isna(row.Trägermaterial):
        pica += "1130 TB-papier\n1130 !040445224!\n"
    else:
        pica += f"1130 {row.Trägermaterial}\n"
    
    if pd.isna(row.Form):
        pica += "1131 !04046198X!\n"
    else:
        pica += f"1131 {row.Form}\n"
    
    pica += "1132 a1-analog;a2-druck;f2-blatt\n"

    if pd.notna(row.Sprache):
        pica += f"1500 /1{row.Sprache}\n"
    
    pica += f"""1700 /1{row.Ländercode}
2105 04,P01
4000 Plakat Nr. {row.Signatur} der Plakatsammlung 1945-1990
4019 Plakat Nr. {row.Signatur} der Plakatsammlung 1945-1990\n"""
    
    if pd.notna(row.Erscheinungsort) and row.Erscheinungsort != '[Ort nicht ermittelbar]':
        pica += f"4030 [{row.Erscheinungsort}] : [Verlag nicht ermittelbar]"
    elif row.Erscheinungsort == '[Ort nicht ermittelbar]' or pd.isna(row.Erscheinungsort):
        pica += f"4030 [Ort nicht ermittelbar] : [Verlag nicht ermittelbar]"
    
    pica += f"""
4060 {row.Anzahl}
4062 Format {row.Format}
4105 !1238824013!\n"""
    
    if pd.notna(row.Anmerkung):
        pica += f"4201 {row.Anmerkung}\n"
        
    if pd.notna(row.Internvermerk):
        pica += f"4700 |BSM|*{row.Internvermerk}\n"
    else:
        pica += f"4700 |BSM|\n"
    
    pica += f"""5590 [Objektgattung]
5590 !041133579!
7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645! 
"""
    be_4801 = list()
    for element in [(row.Zustand), (row.kons_bemerkung)]:
        if type(element) == str:
            be_4801.append(element)
        elif type(element) == float:
            pass
        else:
            be_4801.append(f'FEHLER in {row.Signatur}')
    
    if len(be_4801) > 0:
        pica += f"4801 {'; '.join(be_4801)}\n"
    
    pica += f"""6800 [Provenienz]
6800 !004709608! *1961-1993
6800 Stempel
7100 GS PL {row.Signatur} @ m
7109 !!DBSM/S!! ; GS PL {row.Signatur}
8034 Dieses Plakat ist Teil eines Erschließungsprojekts und derzeit nicht ausleihbar.\n"""
    if pd.notna(row.Altsignatur):
        pica += f"8598 {row.Altsignatur}\n"
    pica += "\t\n"
    return pica

In [4]:
now = datetime.now()
with open(f"../dat/plakate1945-90-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/plakate1945-90-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(datensatz_schreiben(row))
            f2.write(datensatz_schreiben(row))

with open(f"../dat/plakate1945-90-recent-sample.dat", 'w') as f:
    for row in df.sample(5).itertuples():
        f.write(datensatz_schreiben(row))

In [12]:
# teilabzüge zu je 1000 Sätzen schreiben
for i in range(len(df) // 1000 + 1):
     with open(f"../dat/plakate94-90-teil-{i+1}.dat", "w") as f:
      for index, row in df[i*1000:(i+1)*1000].iterrows():
         # print(f"{i*1000}:{(i+1)*1000}")
         f.write(datensatz_schreiben(row))

In [20]:
with open(f"../dat/plakate1945-90-appr-sample.dat", 'w') as f:
    for row in df.sample(750).itertuples():
        f.write(datensatz_schreiben(row))

In [10]:
now = datetime.now()
for row in df.sample(5).itertuples():
    print(datensatz_schreiben(row))

0500 Pa
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1949
1110 *1949$4ezth
1130 TB-papier
1130 !040445224!
1131 !04046198X!
1132 a1-analog;a2-druck;f2-blatt
1500 /1ger
1700 /1XA-DE
2105 04,P01
4000 Plakat Nr. 1945-90/6320 der Plakatsammlung 1945-1990
4019 Plakat Nr. 1945-90/6320 der Plakatsammlung 1945-1990
4030 [SBZ] : [Verlag nicht ermittelbar]
4060 1 ungezähltes Blatt
4062 Format A
4105 !1238824013!
5590 [Objektgattung]
5590 !041133579!
7001 23-08-22 : x
4800 !009033645! 
6800 [Provenienz]
6800 !004709608! *1961-1993
6800 Stempel
7100 GS PL 1945-90/6320 @ m
7109 !!DBSM/S!! ; GS PL 1945-90/6320
8034 Dieses Plakat ist Teil eines Erschließungsprojekts und derzeit nicht ausleihbar.
8598 80/99/P78(3)

0500 Pa
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1946$n[1946?]
1110 1946?*1946$4ezth
1130 TB-papier
1130 !040445224!
1131 !04046198X!
1132 a1-analog;a2-druck;f2-blatt
1500 /1ger
1700 /1XA-DE
2105 04,P01
4000 Plakat Nr

# bekannte Fehler
- 4276 muss durch nachträgliche auswertung von 4201 ausgewertet werden.

# Lösungsansatz für 4276/4201-Abgleich
aus dem nächsten Monatsabzug: Auslesen von IDN, Sig und 4201
simple beziehung: suchen nach signatur in 4201, signatur in gesamttabelle suchen und entsprechende idn in herkunftssatz in 4276 eintragen
mehrfachbeziehung: bereich auflösen und signaturen mehrfach ausschreiben, dann wie simple beziehung handhaben